# cleaning data until 12/15

In [1]:
import pandas as pd
#read csv /Users/jasperhewitt/Desktop/github_repos/Campaigning_101/data/campaign_to_12_15.csv
df=pd.read_csv('/Users/jasperhewitt/Desktop/github_repos/Campaigning_101/data/original_crowdtangle_to_12_15.csv')

In [2]:
df = df.sort_values(by='Post Created')

In [3]:
#create a new column on the left called serial_number that is just a number from 1 to however many rows there are. the column has to be ON THE LEFT


In [4]:
#take out the dates before the other candidates were announced
len(df)

1591

In [5]:
import pandas as pd

# Assuming df is your DataFrame

# Convert the "Post Created Date" column to datetime format
df['Post Created Date'] = pd.to_datetime(df['Post Created Date'])

# Define the target dates for each candidate
hou_dates = pd.to_datetime('2023-05-17')
ke_dates = pd.to_datetime('2023-05-16')
guo_start_date = pd.to_datetime('2023-08-28')
guo_end_date = pd.to_datetime('2023-11-24')

# Filter for 侯友宜: 2023-05-17 and later
df_hou = df[(df['Page Name'] == '侯友宜') & (df['Post Created Date'] >= hou_dates)]

# Filter for 柯文哲: 2023-05-16 and later
df_ke = df[(df['Page Name'] == '柯文哲') & (df['Post Created Date'] >= ke_dates)]

# Filter for 郭台銘: 2023-08-28 to 2023-11-24
df_guo = df[(df['Page Name'] == '郭台銘') & (df['Post Created Date'] >= guo_start_date) & (df['Post Created Date'] <= guo_end_date)]

# Keep all posts for 賴清德 without date filtering
df_lai = df[df['Page Name'] == '賴清德']

# Concatenate the results
filtered_df = pd.concat([df_hou, df_ke, df_guo, df_lai])

# Display the filtered DataFrame
print(filtered_df)

     Page Name User Name      Facebook Id Page Category   
152        侯友宜   houyuih  100044642490075    POLITICIAN  \
727        侯友宜   houyuih  100044642490075    POLITICIAN   
1163       侯友宜   houyuih  100044642490075    POLITICIAN   
483        侯友宜   houyuih  100044642490075    POLITICIAN   
445        侯友宜   houyuih  100044642490075    POLITICIAN   
...        ...       ...              ...           ...   
18         賴清德   chingte  100044573580122    POLITICIAN   
303        賴清德   chingte  100044573580122    POLITICIAN   
939        賴清德   chingte  100044573580122    POLITICIAN   
836        賴清德   chingte  100044573580122    POLITICIAN   
1523       賴清德   chingte  100044573580122    POLITICIAN   

     Page Admin Top Country       Page Description         Page Created   
152                      TW             顧好台灣，人民好康！  2016-05-26 07:40:26  \
727                      TW             顧好台灣，人民好康！  2016-05-26 07:40:26   
1163                     TW             顧好台灣，人民好康！  2016-05-26 07:

In [6]:
filtered_df = filtered_df.sort_values(by='Post Created')

In [7]:
filtered_df.insert(0, 'serial_number', range(1, 1 + len(filtered_df)))

## merge columns and separate metadata from relevant columns

In [8]:
filtered_df.columns

Index(['serial_number', 'Page Name', 'User Name', 'Facebook Id',
       'Page Category', 'Page Admin Top Country', 'Page Description',
       'Page Created', 'Likes at Posting', 'Followers at Posting',
       'Post Created', 'Post Created Date', 'Post Created Time', 'Type',
       'Total Interactions', 'Likes', 'Comments', 'Shares', 'Love', 'Wow',
       'Haha', 'Sad', 'Angry', 'Care', 'Video Share Status', 'Is Video Owner?',
       'Post Views', 'Total Views', 'Total Views For All Crossposts',
       'Video Length', 'URL', 'Message', 'Link', 'Final Link', 'Image Text',
       'Link Text', 'Description', 'Sponsor Id', 'Sponsor Name',
       'Sponsor Category',
       'Total Interactions (weighted  —  Likes 1x Shares 1x Comments 1x Love 1x Wow 1x Haha 1x Sad 1x Angry 1x Care 1x )',
       'Overperforming Score'],
      dtype='object')

In [9]:
#create a new column that combines the contents of Message, Image Text, and Description
#delete the 'index' column
filtered_df['combined_text'] = filtered_df['Message'].fillna('') + filtered_df['Image Text'].fillna('') + filtered_df['Description'].fillna('')

In [10]:
#ok now seperate the data into 2 different dataframes.
#the first df only contains the columns serial_number, User Name, URL, Post Created, combined_text
labelling_df = filtered_df[['serial_number', 'User Name', 'URL', 'Post Created', 'combined_text']]

In [11]:
#later we will have to save both labelling_df and filtered_df to a csv


# keyword search for relevant rows

In [13]:
#check values in Page Name User Name of labelling_df
labelling_df['User Name'].value_counts()

User Name
chingte         577
houyuih         342
DoctorKoWJ      292
TerryGou1018    161
Name: count, dtype: int64

In [ ]:
#create a new column called 'mention_other' that is empty
labelling_df['mention_other'] = ''
#and another one called 'mentioned_other' that is empty
labelling_df['mentioned_other'] = ''
#and another one called 'attack' that is empty
labelling_df['attack'] = ''
#and another one called 'target' that is empty
labelling_df['target'] = ''
#add another one called 'description' that is empty
labelling_df['description'] = ''

In [17]:
# 賴清德
terms_to_check = ["侯", "國民黨", "民眾黨", "柯", "郭", "藍白", "吳", 
                  "馬前總統", "馬英九", "朱", "趙", "賴佩霞", "在野黨", 
                  "藍營", "主流民意大聯盟"] 


# Function to check if any term is present in the 'combined_text' column
def check_terms(text):
    for term in terms_to_check:
        if term in text:
            return 'yes'
    return 'no'

# Filter rows where 'User Name' is 'chingte'
chingte_rows = labelling_df['User Name'] == 'chingte'

# Apply the function to create the 'mention_other' column for 'chingte' rows only
labelling_df.loc[chingte_rows, 'mention_other'] = labelling_df.loc[chingte_rows, 'combined_text'].apply(check_terms)

# Print the updated DataFrame
print(labelling_df)

      serial_number   User Name   
904               1     chingte  \
215               2     chingte   
1580              3     chingte   
104               4     chingte   
1133              5     chingte   
...             ...         ...   
726            1368  DoctorKoWJ   
1045           1369     houyuih   
345            1370  DoctorKoWJ   
1257           1371  DoctorKoWJ   
1523           1372     chingte   

                                                    URL   
904   https://www.facebook.com/100044573580122/posts...  \
215   https://www.facebook.com/100044573580122/posts...   
1580  https://www.facebook.com/100044573580122/posts...   
104   https://www.facebook.com/100044573580122/posts...   
1133  https://www.facebook.com/100044573580122/posts...   
...                                                 ...   
726   https://www.facebook.com/100044470088043/posts...   
1045  https://www.facebook.com/100044642490075/posts...   
345   https://www.facebook.com/100044470088043/p

In [ ]:
import pandas as pd

# Assuming labelling_df is your DataFrame

# List of terms to check for in the 'combined_text' column
terms_to_check = ["侯", "國民黨", "民眾黨", "柯", "郭", "藍白", "吳", 
                  "馬前總統", "馬英九", "朱", "趙", "賴佩霞", "在野黨", 
                  "藍營", "主流民意大聯盟"] 

# Function to check for all terms present in the 'combined_text' column
def check_terms(text):
    hits = []
    for term in terms_to_check:
        if term in text:
            hits.append(term)
    return hits

# Filter rows where 'User Name' is 'chingte'
chingte_rows = labelling_df['User Name'] == 'chingte'

# Apply the function to create the 'mentioned_other' column for 'chingte' rows only
labelling_df.loc[chingte_rows, 'mentioned_other'] = labelling_df.loc[chingte_rows, 'combined_text'].apply(check_terms)

# Convert the list of hits to a comma-separated string
labelling_df.loc[chingte_rows, 'mentioned_other'] = labelling_df.loc[chingte_rows, 'mentioned_other'].apply(lambda x: ', '.join(x) if x else 'no')

# Print the updated DataFrame
print(labelling_df)


In [ ]:
import pandas as pd

# Assuming labelling_df is your DataFrame

# List of terms to check for in the 'combined_text' column
terms_to_check = ["侯", "國民黨", "民眾黨", "柯", "郭", "藍白", "吳", 
                  "馬前總統", "馬英九", "朱", "趙", "賴佩霞", "在野黨", 
                  "藍營", "主流民意大聯盟"]  #吳 is causing a lot of unrelated search results
                                          # I think it is just a very common name lol

# Function to check for all terms present in the 'combined_text' column
def check_terms(text):
    hits = []
    for term in terms_to_check:
        if term in text:
            hits.append(term)
    return hits

# Filter rows where 'User Name' is 'chingte'
chingte_rows = labelling_df['User Name'] == 'chingte'

# Apply the function to create the 'mentioned_other' column for 'chingte' rows only
labelling_df.loc[chingte_rows, 'mentioned_other'] = labelling_df.loc[chingte_rows, 'combined_text'].apply(check_terms)

# Convert the list of hits to a comma-separated string
labelling_df.loc[chingte_rows, 'mentioned_other'] = labelling_df.loc[chingte_rows, 'mentioned_other'].apply(lambda x: ', '.join(x) if x else 'no')

In [19]:
#count number of 'yes' in mention_other column
labelling_df['mention_other'].value_counts()

mention_other
       795
no     513
yes     64
Name: count, dtype: int64